# 사이킷런으로 시작하는 머신러닝 과제
(2조 πsun)

## 1. 개념

### 1-1. 각 문장이 맞는지 틀린지를 T/F로 표시하시오.
1. 사이킷런은 파이썬 머신러닝 라이브러리 중 가장 많이 사용되는 라이브러리이며 다른 머신러닝 패키지도 사이킷런 스타일 API를 지향한다. (  T )
2. 분류는 대표적인 비지도 학습의 방법 중 하나이다. (  F )
3. 지도학습은 결과가 있는 데이터로 학습한 뒤 새로운 데이터를 가지고 결과를 예측하는 방법이다. ( T  )
4. K-Fold 교차 검증은 K개의 데이터 폴드 세트를 만들어 K번만큼 각 fold 세트데 학습과 검증 평가를 반복적으로 수행하는 방법이다. ( T  )
5. Stratified K 폴드는 균형 분포도를 가진 레이블 데이터 집합을 위한 K폴드 방식이다. (  F )

### 1-2. 아래 빈칸을 채우시오.
1. (  하이퍼 파라미터  )란 머신러닝 알고리즘별로 최적의 학습을 위해 직접 입력하는 파라미터를 말한다.
2. 학습 데이터와 테스트 데이터로 분리하는 데 (  test_train_split )함수를 사용한다.
3. 학습 데이터로 학습된 모델의 성능을 평가하려면 (  test data  )가 필요하다.
4. (  estimator  )란 분류 알고리즘을 구현한 클래스 Classifier와 회귀 알고리즘을 구현한 클래스를 Regressor을 합친 클래스를 말한다.
5. (  과적합  )란 학습 데이터에만 과도하게 최적화된 것을 말하며 (  교차 검증  )을 하여 이를 예방한다.

### 1-3. 붓꽃 데이터 세트를 분류하는 예측 프로세스 과정을 순서대로 나열하시오.
1. 모델 학습
2. 평가
3. 데이터 세트 분리
4. 예측 수행    

정답 : (  3  -  1  - 4   -  2  )

## 2. 실습
다음은 위스콘신 유방함 피처들과 악성/음성 레이블 데이터 세트를 분류하는 실습 과제이다. 
아래는 데이터 이해를 위한 변수에 대한 간략한 설명을 담고 있다.     
![위스콘신_데이터_설명](./wisc_cancer.png)

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,cross_val_score,GridSearchCV,train_test_split
from sklearn.datasets import load_breast_cancer
import numpy as np
import pandas as pd

In [3]:
breast_cancer=load_breast_cancer()
features=breast_cancer.data
label=breast_cancer.target
dt_clf=DecisionTreeClassifier(random_state=100)

### 2-1. 다음 데이터를 이용하여 Stratified K폴드와 cross_val_score() 값비교하기

2-1-1. Stratified K폴드이용 (폴드수는 5개로 설정)

In [4]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
cv_accuracy = []
n_iter = 0
for train_index, test_index in skf.split(features,label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    accuracy = np.round(accuracy_score(y_test,pred),4)
    cv_accuracy.append(accuracy)
    
print('교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('평균 검증 정확도:', np.mean(cv_accuracy))

교차 검증별 정확도: [0.913  0.913  0.9027 0.9292 0.9115]
평균 검증 정확도: 0.91388


답:교차 검증별 정확도 : [0.913  0.913  0.9027 0.9292 0.9115]    
평균 검증 정확도 : 0.9139

2-1-2. cross_val_score()이용
(폴드수는 5개로 설정)

In [7]:
val_scores = cross_val_score(dt_clf, features, label, scoring='accuracy', cv=5)
print('교차 검증별 정확도:', np.round(val_scores, 4))
print('평균 검증 정확도:', np.round(np.mean(val_scores), 4))

교차 검증별 정확도: [0.913  0.913  0.9027 0.9292 0.9115]
평균 검증 정확도: 0.9139


답:교차 검증별 정확도 : [0.913  0.913  0.9027 0.9292 0.9115]    
평균 검증 정확도 : 0.9139

### 2-2. GridSearchCV를 이용하여 정확도 구하기
(max_depth는 1,2,3,4,5/min_samples_split은 2,3,4,5/ 폴드수는 6개/TEST_SIZE는 0.1로 각각 설정)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.1, random_state=200)

dt_clf=DecisionTreeClassifier(random_state=100)
parameters = {'max_depth':[1,2,3,4,5], 'min_samples_split':[2,3,4,5]}

grid_dt_clf = GridSearchCV(dt_clf,param_grid = parameters, cv=6, refit=True)

# 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습 및 평가
grid_dt_clf.fit(X_train, y_train)

#GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dt_clf.cv_results_)
scores_df


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002321,4.684892e-04,0.000830,0.000371,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.896552,0.905882,0.929412,0.894118,0.917647,0.905882,0.908203,0.012136,13
1,0.001663,7.448762e-04,0.000665,0.000470,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.896552,0.905882,0.929412,0.894118,0.917647,0.905882,0.908203,0.012136,13
2,0.001163,3.716203e-04,0.000166,0.000372,1,4,"{'max_depth': 1, 'min_samples_split': 4}",0.896552,0.905882,0.929412,0.894118,0.917647,0.905882,0.908203,0.012136,13
3,0.001665,4.716124e-04,0.000166,0.000372,1,5,"{'max_depth': 1, 'min_samples_split': 5}",0.896552,0.905882,0.929412,0.894118,0.917647,0.905882,0.908203,0.012136,13
4,0.002825,3.721183e-04,0.000000,0.000000,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.908046,0.941176,0.894118,0.894118,0.917647,0.894118,0.908203,0.017150,13
5,0.001994,3.576279e-07,0.000499,0.000499,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.908046,0.941176,0.894118,0.894118,0.917647,0.894118,0.908203,0.017150,13
6,0.002161,3.716739e-04,0.000333,0.000470,2,4,"{'max_depth': 2, 'min_samples_split': 4}",0.908046,0.941176,0.894118,0.894118,0.917647,0.894118,0.908203,0.017150,13
7,0.002327,4.703871e-04,0.000166,0.000372,2,5,"{'max_depth': 2, 'min_samples_split': 5}",0.908046,0.941176,0.894118,0.894118,0.917647,0.894118,0.908203,0.017150,13
8,0.003158,3.716384e-04,0.000166,0.000372,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.896552,0.952941,0.870588,0.905882,0.952941,0.905882,0.914062,0.029832,9
9,0.003158,3.721713e-04,0.000166,0.000372,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.896552,0.952941,0.870588,0.905882,0.952941,0.905882,0.914062,0.029832,9


답 : GridSearchCV 최적 파라미터: {'max_depth': 4, 'min_samples_split': 4}    
GridSearchV 최고정확도 : 0.9375    
테스트 데이터 세트 정확도 : 0.9298

In [15]:
print('GridSearchCV 최적 파라미터:',grid_dt_clf.best_params_)

GridSearchCV 최적 파라미터: {'max_depth': 5, 'min_samples_split': 2}


In [18]:
print('GridSearchCV 최고 정확도:{:.4f}'.format(grid_dt_clf.best_score_))

GridSearchCV 최고 정확도:0.9375


In [21]:
estimator = grid_dt_clf.best_estimator_
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도:{0:4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도:0.912281
